In [1]:
#%%
from array import array
from cmath import nan
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
import torchvision
import torchvision.transforms as transforms
from itertools import chain
from sklearn import metrics as met
import pickle
import icecream as ic
import torchsummary
import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
import util
from torchmetrics import Accuracy
from torch.utils.data.datapipes.datapipe import _IterDataPipeSerializationWrapper, _MapDataPipeSerializationWrapper
from torch.utils.data import default_convert
from tqdm import tqdm
import time

/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
train_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/gene_seq_train.csv')
train_data = train_data.iloc[:, 1:]
train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/res_train.csv')

In [16]:
from random import choice
def rand_seq_gen(length):
   DNA=""
   for count in range(length):
      DNA+=choice("CGTA")
   return DNA

In [20]:
#generating random sample in the training dataset
from random import seed
from random import randint
from tqdm import tqdm

rand_size = int(train_data.shape[0]*0.05)
random_df = pd.DataFrame(columns=[train_data.columns])
rand_res_df = pd.DataFrame(0, index=np.arange(rand_size), columns=[train_target.columns])

#get mean length of each loci
loci_len = {}
for x in train_data.columns:
    loci_len[x] = train_data[x].str.split("").str.len().mean().round()

#create instances with random sequences
target_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #dr to be added in corresponding to rand seq

rand_size = int(len(train_data)*0.05) #amount of rand seq
train_data_rand = train_data.copy()
train_target_rand = train_target.copy()
seed(42)
# generate some integers for insert position
for _ in tqdm(range(rand_size)):
	insert_pos = randint(0, len(train_data))
	list_ = [] #creating rand seq
	for k, v in loci_len.items():
		list_.append(rand_seq_gen(int(v)))

	train_data_rand = pd.DataFrame(np.insert(train_data_rand.values, insert_pos, list_, axis=0)) 
	train_target_rand = pd.DataFrame(np.insert(train_target_rand.values, insert_pos, target_list, axis=0)) 
	# for x in range(rand_size):
	# 	list_ = []
	# 	for k, v in loci_len.items():
	# 		list_.append(rand_seq_gen(int(v)))
	# 	random_df = random_df.append(pd.DataFrame([list_], columns=[train_data.columns]), ignore_index=True)
train_data_rand.columns = train_data.columns
train_target_rand.columns = train_target.columns

100%|██████████| 552/552 [00:19<00:00, 27.65it/s]


In [21]:
from pathlib import Path  
filepath = Path('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/seq_5percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
train_data_rand.to_csv(filepath, index=False, index_label=False)

filepath = Path('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/res_5percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
train_target_rand.to_csv(filepath, index=False, index_label=False)